In [5]:
# =====================================================================================
# CÓDIGO CORREGIDO PARA RECONOCIMIENTO DE GESTOS EN GOOGLE COLAB
# =====================================================================================

# 1. INSTALACIÓN DE LIBRERÍAS
# Las librerías principales ya vienen preinstaladas, pero aseguramos MediaPipe.
print("Instalando librerías...")
!pip install mediapipe opencv-python > /dev/null

# 2. IMPORTACIONES NECESARIAS
import cv2
import mediapipe as mp
import numpy as np
from IPython.display import display, Javascript # Para manejar la webcam
from base64 import b64decode
from io import BytesIO
from PIL import Image

# 3. CONFIGURACIÓN DE MEDIAPIPE
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# --- Función para Clasificar Gestos ---
def classify_gesture(hand_landmarks):
    """Clasifica el gesto basándose en la posición de los dedos."""

    # Puntos de referencia de las puntas (TIP)
    tip_ids = [mp_hands.HandLandmark.INDEX_FINGER_TIP,
               mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
               mp_hands.HandLandmark.RING_FINGER_TIP,
               mp_hands.HandLandmark.PINKY_TIP]

    fingers_up = 0

    # Chequeo del Pulgar (asumiendo mano derecha: TIP_4 está a la izquierda del MCP_2)
    # Nota: Este chequeo es simple y puede fallar con la mano izquierda o ángulos raros.
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    thumb_ip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP]

    # Si la punta del pulgar (TIP) está más allá de la articulación (IP) en el eje X (para el pulgar hacia afuera)
    if thumb_tip.x < thumb_ip.x:
        fingers_up += 1

    # Chequeo de los demás dedos (Índice, Medio, Anular, Meñique)
    for id in tip_ids:
        tip = hand_landmarks.landmark[id]
        pip = hand_landmarks.landmark[id - 2]

        # Si la punta (TIP) está más arriba (menor valor Y) que la articulación (PIP), el dedo está levantado.
        if tip.y < pip.y:
            fingers_up += 1

    # Clasificación de Gestos
    if fingers_up == 5:
        return "🖐️ Mano Abierta"
    elif fingers_up == 1 and hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y:
        return "👆 Apuntando"
    elif fingers_up == 0:
        return "✊ Puño Cerrado"
    elif fingers_up == 2 and hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y and hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y:
        return "✌️ Victoria"
    else:
        return f"Dedos Arriba: {fingers_up}"

# --- CÓDIGO DE MANEJO DE WEBCAM EN COLAB ---

def video_stream():
  """Define las funciones JavaScript para controlar y tomar fotos de la cámara."""
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;

    var shutdown = false;

    async function startCamera() {
      if (stream) return;

      div = document.createElement('div');
      document.body.appendChild(div);

      video = document.createElement('video');
      video.style.display = 'block';
      stream = await navigator.mediaDevices.getUserMedia({video: true});
      div.appendChild(video);

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640;
      captureCanvas.height = 480;

      imgElement = document.createElement('img');
      imgElement.src = 'data:image/jpeg;base64,' + '';
      div.appendChild(imgElement);
    }

    async function takePhoto(quality = 0.8) {
      if (shutdown) return;
      var ctx = captureCanvas.getContext('2d');
      ctx.drawImage(video, 0, 0, captureCanvas.width, captureCanvas.height);
      var data = captureCanvas.toDataURL('image/jpeg', quality);

      // La clave del fix: Actualiza la imagen visible en Colab (opcional, pero útil)
      imgElement.src = data;

      // Devuelve solo el string de base64
      return data.substring(data.indexOf(',')+1);
    }

    function stopCamera() {
      if (stream) {
        stream.getTracks().forEach(track => track.stop());
      }
      shutdown = true;
      if(div) {
        document.body.removeChild(div);
      }
    }
    ''')
  display(js)

def get_frame():
  """Llama a la función JS y decodifica la data en una imagen OpenCV."""
  # La función eval(display(...).data) obtiene la data de la imagen de JS
  data = eval(display(Javascript('takePhoto(0.5)'), raw=True).data)

  # Decodifica la base64 y la convierte a formato OpenCV (BGR)
  img_data = b64decode(data)
  img_np = np.frombuffer(img_data, np.uint8)
  frame = cv2.imdecode(img_np, cv2.IMREAD_COLOR)
  return frame

# =====================================================================================
# 4. INICIO Y BUCLE PRINCIPAL DE RECONOCIMIENTO
# =====================================================================================

# 4.1. Iniciar el stream de video en el navegador
video_stream()
display(Javascript('startCamera()'))
print("Cámara iniciada. Espera 3 segundos y haz tu gesto. La ejecución tomará 100 frames.")

# 4.2. Bucle principal
FRAME_COUNT = 100 # Número de frames a capturar
for i in range(FRAME_COUNT):
    try:
        frame = get_frame()
    except Exception as e:
        # Esto maneja errores de inicialización o si la cámara no está lista
        # print(f"Error en frame {i}: {e}. Intentando de nuevo...")
        cv2.waitKey(100) # Pequeña espera
        continue

    # Pre-procesamiento
    frame = cv2.flip(frame, 1) # Espejo
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Procesamiento de MediaPipe
    results = hands.process(rgb_frame)

    current_gesture = "Esperando Gesto..."

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Dibujar los landmarks
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2)
            )

            # Clasificar el gesto
            current_gesture = classify_gesture(hand_landmarks)

    # Mostrar el resultado en el frame de OpenCV
    cv2.putText(frame, current_gesture, (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)

    # Mostrar el frame procesado. En Colab, cv2_imshow es la forma de mostrar.
    # Nota: Colab solo muestra el ÚLTIMO frame en la salida de la celda, pero el JS actualiza la imagen en vivo.
    # cv2_imshow(frame)

    cv2.waitKey(1)

# 4.3. Limpieza al finalizar
print("Reconocimiento terminado. Deteniendo la cámara...")
# Detener el stream de la cámara del navegador
display(Javascript('stopCamera()'))

Instalando librerías...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Cámara iniciada. Espera 3 segundos y haz tu gesto. La ejecución tomará 100 frames.
Reconocimiento terminado. Deteniendo la cámara...


<IPython.core.display.Javascript object>